## 1.2.6 例:素数判定

ここでは、整数$n$の素数性をチェックする2つの方法を紹介する。  
- 増加オーダーが$\Theta(\sqrt{n})$であるアルゴリズム  
- 増加オーダーが$\Theta(\log{n})$である"確率的"なアルゴリズム  

#### 約数を探す
ある数値が素数であるかテストする⽅法のひとつとして、  
その数値の約数を探すというものがある。  
次のプログラムは、$2$から始まる⼀連の数字$n$について割り切れるかどうかを調べるという素直な⽅法で、約数を探している。

In [1]:
(define (smallest-divisor n)
  (find-divisor n 2)
  )
(define (find-divisor n test-divisor)
  (cond ((> (square test-divisor) n) n)
        ((divides? test-divisor n) test-divisor)
        (else (find-divisor n (+ test-divisor 1)))
    )
  )
(define (divides? a b)
  (= (remainder b a) 0)
  )
(define (square x) (* x x))

数値が素数であるかは次のようにテストできる:$n$は、$n$⾃⾝がその最⼩の約数である場合、かつその場合に限り、素数である。

In [2]:
(define (prime? n)
  (= n (smallest-divisor n))
  )
; 動作確認
(display (prime? 27))
(newline)
(display (prime? 109))
(newline)

False
True


find-divisorの終了条件は、もし$n$が素数でないならば、  
それは$\sqrt{n}$以下の約数を持つという事実に基づいている。  
つまり、このアルゴリズムは、$1$から$\sqrt{n}$までの約数についてだけテストすればよい。よって、このプログラムの必要なステップ数の増加オーダーは$\Theta(\sqrt{n})$となる。

#### フェルマーテスト
フェルマーの小定理(整数論の結果)に基づいたステップ数の増加オーダーが$\Theta(\log{n})$であるアルゴリズムを紹介する。

##### フェルマーの小定理
$n$が素数で、$a$が$n$より⼩さい任意の正の整数であるとき、  
$a$の$n$乗は法$n$に関して$a$と合同である。

つまり、素数$n$と$n>a>0$となる任意の整数$a$に対して、    
$mod(a,n) \equiv mod(a^n,n)$  
が成り立つ。

In [18]:
(define (expmod base exp m)
  (cond ((= exp 0) 1)
        ((even? exp)(remainder (square (expmod base (/ exp 2) m)) m))
        (else (remainder (* base (expmod base (- exp 1) m)) m)))
  )

In [19]:
(define (fermat-test n)
  (define (try-it a)
    (= (expmod a n n) a))
  (try-it (+ 1 (random (- n 1))))
  )

In [20]:
(define (fast-prime? n times)
  (cond ((= times 0) #t)
        ((fermat-test n) (fast-prime? n (- times 1)))
        (else #f))
  )

In [21]:
(fast-prime? 1039 10)

#t